# 17 - DuckDB + SciPy: Matemáticas y Series de Tiempo Básicas

## 🎯 Objetivos
- Operaciones matemáticas con SciPy
- Optimización numérica
- Estadística inferencial
- Análisis de series temporales básicas
- Integración y diferenciación numérica
- Procesamiento de señales
- Análisis con DuckDB

## 📚 Tecnologías
- **DuckDB**: Procesamiento y análisis de datos
- **SciPy**: Computación científica
- **NumPy**: Operaciones numéricas
- **Pandas**: Manipulación de datos

## ⭐ Complejidad: Básico/Intermedio

## 1. Instalación y Setup

In [ ]:
# Instalar dependencias
!pip install duckdb pandas numpy scipy matplotlib seaborn plotly statsmodels -q

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# SciPy modules
import scipy
from scipy import stats
from scipy import optimize
from scipy import integrate
from scipy import interpolate
from scipy import signal
from scipy import fft
from scipy.stats import norm, t, chi2, f_oneway, pearsonr, spearmanr

# Config
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"✅ SciPy version: {scipy.__version__}")
print(f"✅ NumPy version: {np.__version__}")
print(f"✅ DuckDB version: {duckdb.__version__}")

## 2. Estadística Básica con SciPy y DuckDB

In [ ]:
# Conectar a DuckDB
con = duckdb.connect(':memory:')

# Generar datos sintéticos
np.random.seed(42)

# Ventas de diferentes tiendas
n_samples = 1000
data = {
    'store_id': np.repeat(['Store_A', 'Store_B', 'Store_C', 'Store_D'], n_samples // 4),
    'sales': np.concatenate([
        np.random.normal(1000, 200, n_samples // 4),  # Store A
        np.random.normal(1200, 180, n_samples // 4),  # Store B
        np.random.normal(950, 250, n_samples // 4),   # Store C
        np.random.normal(1100, 190, n_samples // 4)   # Store D
    ]),
    'customers': np.random.poisson(50, n_samples),
    'avg_ticket': np.random.lognormal(3, 0.5, n_samples)
}

df = pd.DataFrame(data)

print(f"📊 Dataset generado: {len(df)} registros")
print(f"\n📊 Primeras filas:")
print(df.head())

# Estadísticas con DuckDB
stats_db = con.execute("""
    SELECT 
        store_id,
        COUNT(*) as n,
        ROUND(AVG(sales), 2) as mean_sales,
        ROUND(STDDEV(sales), 2) as std_sales,
        ROUND(MIN(sales), 2) as min_sales,
        ROUND(MAX(sales), 2) as max_sales,
        ROUND(MEDIAN(sales), 2) as median_sales
    FROM df
    GROUP BY store_id
    ORDER BY mean_sales DESC
""").df()

print(f"\n📊 Estadísticas por tienda (DuckDB):")
print(stats_db)

## 3. Tests Estadísticos con SciPy

In [ ]:
# Test de normalidad
print("📊 TEST DE NORMALIDAD (Shapiro-Wilk)\n")

for store in df['store_id'].unique():
    sales = df[df['store_id'] == store]['sales']
    stat, p_value = stats.shapiro(sales)
    
    is_normal = "✅ Normal" if p_value > 0.05 else "❌ No normal"
    print(f"{store}: W={stat:.4f}, p-value={p_value:.4f} {is_normal}")

# Test de varianza (Levene)
print(f"\n📊 TEST DE VARIANZA HOMOGÉNEA (Levene)\n")

stores_data = [df[df['store_id'] == store]['sales'].values for store in df['store_id'].unique()]
stat, p_value = stats.levene(*stores_data)

print(f"Estadístico: {stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Conclusión: {'Varianzas homogéneas ✅' if p_value > 0.05 else 'Varianzas heterogéneas ❌'}")

# ANOVA
print(f"\n📊 ANÁLISIS DE VARIANZA (ANOVA)\n")

f_stat, p_value = f_oneway(*stores_data)

print(f"F-estadístico: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Conclusión: {'Medias significativamente diferentes ✅' if p_value < 0.05 else 'No hay diferencia significativa ❌'}")

# Test T de dos muestras
print(f"\n📊 TEST T (Store A vs Store B)\n")

store_a = df[df['store_id'] == 'Store_A']['sales']
store_b = df[df['store_id'] == 'Store_B']['sales']

t_stat, p_value = stats.ttest_ind(store_a, store_b)

print(f"t-estadístico: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Conclusión: {'Medias diferentes ✅' if p_value < 0.05 else 'Medias similares ❌'}")

# Correlación
print(f"\n📊 CORRELACIONES\n")

# Pearson
corr_p, p_value_p = pearsonr(df['sales'], df['customers'])
print(f"Pearson (sales vs customers): r={corr_p:.4f}, p={p_value_p:.4f}")

# Spearman
corr_s, p_value_s = spearmanr(df['sales'], df['customers'])
print(f"Spearman (sales vs customers): ρ={corr_s:.4f}, p={p_value_s:.4f}")

## 4. Optimización Numérica

In [ ]:
# Ejemplo: Optimizar precio para maximizar ingresos
# Modelo: Revenue = price * demand(price)
# donde demand(price) = 1000 - 5 * price

def revenue(price):
    """Función de ingresos a maximizar"""
    demand = 1000 - 5 * price
    return -(price * demand)  # Negativo porque minimize busca mínimo

# Optimizar
result = optimize.minimize_scalar(revenue, bounds=(0, 200), method='bounded')

optimal_price = result.x
max_revenue = -result.fun
optimal_demand = 1000 - 5 * optimal_price

print("📊 OPTIMIZACIÓN DE PRECIO\n")
print(f"Precio óptimo: ${optimal_price:.2f}")
print(f"Demanda esperada: {optimal_demand:.0f} unidades")
print(f"Ingresos máximos: ${max_revenue:,.2f}")

# Visualizar
prices = np.linspace(0, 200, 100)
revenues = [price * (1000 - 5 * price) for price in prices]

plt.figure(figsize=(10, 6))
plt.plot(prices, revenues, linewidth=2, label='Ingresos')
plt.axvline(optimal_price, color='r', linestyle='--', label=f'Precio óptimo: ${optimal_price:.2f}')
plt.axhline(max_revenue, color='g', linestyle='--', alpha=0.5, label=f'Ingresos máx: ${max_revenue:,.0f}')
plt.xlabel('Precio ($)')
plt.ylabel('Ingresos ($)')
plt.title('Optimización de Precio')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Ejemplo 2: Ajuste de curva
print(f"\n📊 AJUSTE DE CURVA (Curve Fitting)\n")

# Datos de ejemplo
x_data = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y_data = np.array([1.0, 2.3, 4.2, 7.1, 11.5, 17.8, 25.9, 35.2, 46.3, 59.0, 73.1])

# Función cuadrática
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

# Ajustar
params, covariance = optimize.curve_fit(quadratic, x_data, y_data)

print(f"Parámetros ajustados:")
print(f"   a = {params[0]:.4f}")
print(f"   b = {params[1]:.4f}")
print(f"   c = {params[2]:.4f}")

# Visualizar
x_fit = np.linspace(0, 10, 100)
y_fit = quadratic(x_fit, *params)

plt.figure(figsize=(10, 6))
plt.scatter(x_data, y_data, s=100, alpha=0.6, label='Datos reales')
plt.plot(x_fit, y_fit, 'r-', linewidth=2, label=f'Ajuste: {params[0]:.2f}x² + {params[1]:.2f}x + {params[2]:.2f}')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Ajuste de Curva Cuadrática')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Integración y Diferenciación Numérica

In [ ]:
# Integración numérica
print("📊 INTEGRACIÓN NUMÉRICA\n")

# Función a integrar: f(x) = x^2
def f(x):
    return x**2

# Integral definida de 0 a 3
result, error = integrate.quad(f, 0, 3)

print(f"∫₀³ x² dx = {result:.4f} (error: {error:.2e})")
print(f"Valor analítico: {3**3 / 3:.4f}")

# Integración de datos discretos
x = np.linspace(0, 10, 100)
y = np.sin(x)

# Método trapecio
integral_trapz = integrate.trapezoid(y, x)
# Método Simpson
integral_simps = integrate.simpson(y, x)

print(f"\nIntegración de sin(x) de 0 a 10:")
print(f"   Trapecio: {integral_trapz:.4f}")
print(f"   Simpson: {integral_simps:.4f}")
print(f"   Analítico: {-np.cos(10) + np.cos(0):.4f}")

# Diferenciación numérica
print(f"\n📊 DIFERENCIACIÓN NUMÉRICA\n")

# Calcular derivada de sin(x)
dx = x[1] - x[0]
dy_dx = np.gradient(y, dx)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(x, y, label='f(x) = sin(x)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Función Original')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(x, dy_dx, label="f'(x) numérica", linewidth=2)
plt.plot(x, np.cos(x), '--', label="f'(x) = cos(x) analítica", linewidth=2)
plt.xlabel('x')
plt.ylabel("y'")
plt.title('Derivada')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Series de Tiempo Básicas

In [ ]:
# Generar serie temporal sintética
np.random.seed(42)

n_days = 365
dates = pd.date_range(start='2024-01-01', periods=n_days, freq='D')

# Componentes
trend = np.linspace(100, 150, n_days)
seasonal = 20 * np.sin(2 * np.pi * np.arange(n_days) / 365)
noise = np.random.normal(0, 5, n_days)

sales_ts = trend + seasonal + noise

ts_df = pd.DataFrame({
    'date': dates,
    'sales': sales_ts,
    'day_of_week': dates.dayofweek,
    'month': dates.month,
    'day_of_year': dates.dayofyear
})

print(f"📊 Serie temporal generada: {len(ts_df)} días")
print(f"\n📊 Estadísticas:")
print(ts_df['sales'].describe())

# Análisis con DuckDB
monthly_stats = con.execute("""
    SELECT 
        month,
        COUNT(*) as days,
        ROUND(AVG(sales), 2) as avg_sales,
        ROUND(MIN(sales), 2) as min_sales,
        ROUND(MAX(sales), 2) as max_sales,
        ROUND(STDDEV(sales), 2) as std_sales
    FROM ts_df
    GROUP BY month
    ORDER BY month
""").df()

print(f"\n📊 Estadísticas mensuales:")
print(monthly_stats)

## 7. Análisis Espectral con FFT

In [ ]:
# Transformada de Fourier para detectar periodicidades
print("📊 ANÁLISIS DE FRECUENCIAS (FFT)\n")

# FFT
fft_values = fft.fft(sales_ts)
fft_freq = fft.fftfreq(len(sales_ts), d=1)  # frecuencias (1 día)

# Magnitud
fft_magnitude = np.abs(fft_values)

# Solo frecuencias positivas
positive_freq_idx = fft_freq > 0

# Encontrar picos
peaks, _ = signal.find_peaks(fft_magnitude[positive_freq_idx], height=100)

print(f"Periodicidades detectadas:")
for peak in peaks[:5]:  # Top 5
    freq = fft_freq[positive_freq_idx][peak]
    period = 1 / freq
    magnitude = fft_magnitude[positive_freq_idx][peak]
    print(f"   Período: {period:.1f} días (magnitud: {magnitude:.2f})")

# Visualizar
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Serie temporal
axes[0].plot(ts_df['date'], ts_df['sales'], linewidth=1)
axes[0].set_title('Serie Temporal Original')
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('Ventas')
axes[0].grid(True, alpha=0.3)

# Espectro de frecuencias
axes[1].plot(fft_freq[positive_freq_idx], fft_magnitude[positive_freq_idx], linewidth=1)
axes[1].set_title('Espectro de Frecuencias')
axes[1].set_xlabel('Frecuencia (1/día)')
axes[1].set_ylabel('Magnitud')
axes[1].set_xlim(0, 0.1)  # Zoom en frecuencias bajas
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Suavizado y Filtrado de Señales

In [ ]:
# Moving average con SciPy
window_size = 7
weights = np.ones(window_size) / window_size
sales_ma = np.convolve(sales_ts, weights, mode='valid')

# Suavizado Savitzky-Golay
sales_sg = signal.savgol_filter(sales_ts, window_length=21, polyorder=3)

# Filtro Gaussiano
from scipy.ndimage import gaussian_filter1d
sales_gaussian = gaussian_filter1d(sales_ts, sigma=3)

# Visualizar
plt.figure(figsize=(14, 8))

plt.plot(ts_df['date'], sales_ts, alpha=0.5, label='Original', linewidth=1)
plt.plot(ts_df['date'][window_size-1:], sales_ma, label=f'Moving Average ({window_size} días)', linewidth=2)
plt.plot(ts_df['date'], sales_sg, label='Savitzky-Golay', linewidth=2)
plt.plot(ts_df['date'], sales_gaussian, label='Gaussiano', linewidth=2)

plt.title('Técnicas de Suavizado')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Comparación de suavizado:")
print(f"   Original - Std: {np.std(sales_ts):.2f}")
print(f"   Moving Avg - Std: {np.std(sales_ma):.2f}")
print(f"   Savitzky-Golay - Std: {np.std(sales_sg):.2f}")
print(f"   Gaussiano - Std: {np.std(sales_gaussian):.2f}")

## 9. Interpolación

In [ ]:
# Simular datos con valores faltantes
x_sparse = np.array([0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 364])
y_sparse = sales_ts[x_sparse]

# Diferentes métodos de interpolación
x_dense = np.arange(365)

# Lineal
f_linear = interpolate.interp1d(x_sparse, y_sparse, kind='linear')
y_linear = f_linear(x_dense)

# Cúbica
f_cubic = interpolate.interp1d(x_sparse, y_sparse, kind='cubic')
y_cubic = f_cubic(x_dense)

# Spline
tck = interpolate.splrep(x_sparse, y_sparse, s=0)
y_spline = interpolate.splev(x_dense, tck)

# Visualizar
plt.figure(figsize=(14, 6))

plt.scatter(x_sparse, y_sparse, s=100, c='red', zorder=3, label='Datos originales (dispersos)')
plt.plot(x_dense, sales_ts, 'k--', alpha=0.3, label='Serie completa (referencia)', linewidth=1)
plt.plot(x_dense, y_linear, label='Interpolación Lineal', linewidth=2)
plt.plot(x_dense, y_cubic, label='Interpolación Cúbica', linewidth=2)
plt.plot(x_dense, y_spline, label='Spline', linewidth=2)

plt.title('Métodos de Interpolación')
plt.xlabel('Día del año')
plt.ylabel('Ventas')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Errores de interpolación
print("📊 Error de interpolación (RMSE vs serie completa):")
print(f"   Lineal: {np.sqrt(np.mean((y_linear - sales_ts)**2)):.2f}")
print(f"   Cúbica: {np.sqrt(np.mean((y_cubic - sales_ts)**2)):.2f}")
print(f"   Spline: {np.sqrt(np.mean((y_spline - sales_ts)**2)):.2f}")

## 10. Descomposición de Series Temporales

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Crear serie con índice de fecha
ts_series = pd.Series(sales_ts, index=dates)

# Descomponer
decomposition = seasonal_decompose(ts_series, model='additive', period=30)

# Visualizar
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

decomposition.observed.plot(ax=axes[0], title='Serie Original')
axes[0].set_ylabel('Observado')

decomposition.trend.plot(ax=axes[1], title='Tendencia')
axes[1].set_ylabel('Tendencia')

decomposition.seasonal.plot(ax=axes[2], title='Estacionalidad')
axes[2].set_ylabel('Estacional')

decomposition.resid.plot(ax=axes[3], title='Residuos')
axes[3].set_ylabel('Residuos')
axes[3].set_xlabel('Fecha')

plt.tight_layout()
plt.show()

# Estadísticas de componentes
print("📊 Estadísticas de componentes:")
print(f"   Tendencia - Rango: [{decomposition.trend.min():.2f}, {decomposition.trend.max():.2f}]")
print(f"   Estacional - Amplitud: {decomposition.seasonal.max() - decomposition.seasonal.min():.2f}")
print(f"   Residuos - Std: {decomposition.resid.std():.2f}")

## 11. Análisis con DuckDB: Queries Avanzados

In [ ]:
# Agregar componentes descompuestos a DataFrame
ts_df['trend'] = decomposition.trend
ts_df['seasonal'] = decomposition.seasonal
ts_df['residual'] = decomposition.resid

# Análisis con SQL
trend_analysis = con.execute("""
    WITH monthly_trends AS (
        SELECT 
            month,
            AVG(sales) as avg_sales,
            AVG(trend) as avg_trend,
            AVG(seasonal) as avg_seasonal,
            STDDEV(residual) as std_residual
        FROM ts_df
        WHERE trend IS NOT NULL
        GROUP BY month
    )
    SELECT 
        month,
        ROUND(avg_sales, 2) as avg_sales,
        ROUND(avg_trend, 2) as avg_trend,
        ROUND(avg_seasonal, 2) as avg_seasonal,
        ROUND(std_residual, 2) as volatility,
        ROUND((avg_trend - LAG(avg_trend) OVER (ORDER BY month)) / LAG(avg_trend) OVER (ORDER BY month) * 100, 2) as trend_growth_pct
    FROM monthly_trends
    ORDER BY month
""").df()

print("📊 Análisis de tendencias mensuales:")
print(trend_analysis)

# Detectar anomalías con SQL
anomalies = con.execute("""
    WITH stats AS (
        SELECT 
            AVG(residual) as mean_resid,
            STDDEV(residual) as std_resid
        FROM ts_df
        WHERE residual IS NOT NULL
    )
    SELECT 
        date,
        sales,
        residual,
        ROUND((residual - s.mean_resid) / s.std_resid, 2) as z_score
    FROM ts_df, stats s
    WHERE ABS((residual - s.mean_resid) / s.std_resid) > 2.5
    AND residual IS NOT NULL
    ORDER BY ABS((residual - s.mean_resid) / s.std_resid) DESC
""").df()

print(f"\n📊 Anomalías detectadas (|z-score| > 2.5): {len(anomalies)}")
if len(anomalies) > 0:
    print(anomalies.head(10))

## 12. Resumen

In [ ]:
print("🎉 RESUMEN: SciPy + DuckDB para Matemáticas y Series Temporales")
print("=" * 70)

print("\n✅ CONCEPTOS CUBIERTOS:")
print("   1. Estadística inferencial (tests t, ANOVA, correlaciones)")
print("   2. Optimización numérica (minimización, ajuste de curvas)")
print("   3. Integración y diferenciación numérica")
print("   4. Análisis de series temporales")
print("   5. Análisis espectral (FFT)")
print("   6. Suavizado y filtrado de señales")
print("   7. Interpolación")
print("   8. Descomposición de series")
print("   9. Detección de anomalías")
print("   10. Integración DuckDB para análisis SQL")

print("\n💡 APLICACIONES PRÁCTICAS:")
print("   - Forecasting de demanda")
print("   - Optimización de precios")
print("   - Control de calidad (detección de anomalías)")
print("   - Análisis financiero")
print("   - Procesamiento de señales")
print("   - Análisis de tendencias")

print("\n📚 BIBLIOTECAS CLAVE:")
print("   scipy.stats - Estadística")
print("   scipy.optimize - Optimización")
print("   scipy.integrate - Integración")
print("   scipy.signal - Procesamiento de señales")
print("   scipy.fft - Transformada de Fourier")
print("   scipy.interpolate - Interpolación")

con.close()
print("\n✅ Conexión DuckDB cerrada")
print("\n" + "=" * 70)